# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Low Rank Adaptation (LoRA)
* Model: GPT-2
* Evaluation approach:  Hugging Face Trainer with accuracy as the evaluation metric
* Fine-tuning dataset: AG News

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
# Load the dataset

from transformers import AutoModelForSequenceClassification, AutoTokenizer, GPT2Config, GPT2ForSequenceClassification
from datasets import load_dataset
from transformers import Trainer, TrainingArguments, EvalPrediction

# Load the AG News dataset
dataset = load_dataset("ag_news", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"] = dataset["train"].select(range(0, int(0.01 * len(dataset['train']))))
dataset["train"]

In [ ]:
dataset["test"] = dataset["test"].select(range(0, int(0.1 * len(dataset['train']))))
dataset["test"]

In [ ]:
dataset["train"][0]

In [ ]:
# Preprocess the dataset

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", num_labels=4)

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True), batched=True
    )

tokenized_dataset["train"]

In [ ]:
# Set up the model

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=4,
    id2label={0: "world", 1: "sports", 2: "business", 3: "sci/tech"},
    label2id={"world": 0, "sports": 1, "business": 2, "sci/tech": 3},
)

print(model)

In [ ]:
# Train the model

import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./results/ag_news",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=10,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
# Show the performance of the model on the test set
trainer.evaluate()

# Save a Trained Model
model.save_pretrained('model/base_model')

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# Create a PEFT Config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_lin", "v_lin"],
    task_type=TaskType.SEQ_CLS,
)

# training and testing sets
train_lora = tokenized_dataset['train'].rename_column('label', 'labels')
test_lora = tokenized_dataset['test'].rename_column('label', 'labels')

In [ ]:
# Create a PEFT Model
peft_model = get_peft_model(model, lora_config)

In [ ]:
# Train the PEFT model

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

peft_trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./results/ag_news_peft",
        learning_rate=1e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=10,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

peft_trainer.train()

In [ ]:
# Final evaluation after fine-tuning
eval_results = peft_trainer.evaluate()
print(f"Final evaluation results after fine-tuning: {eval_results}")

# Save a Trained PEFT Model
peft_model.save_pretrained('model/peft_model')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
# Load the model
model = AutoModelForSequenceClassification.from_pretrained("model/base_model", num_labels=4)
print(model)

In [ ]:
# Load the PFET model
peft_model = AutoModelForSequenceClassification.from_pretrained("model/peft_model", num_labels=4)
print(peft_model)

In [ ]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
      [0, 1, 3, 9, 10, 15, 18,  22, 30, 31, 38, 43, 45, 50, 61, 70, 72, 80, 84, 89, 91]
)

results = trainer.predict(items_for_manual_review)

df = pd.DataFrame(
    {
        "sms": [item["text"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

In [ ]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
   [0, 1, 3, 9, 10, 15, 18,  22, 30, 31, 38, 43, 45, 50, 61, 70, 72, 80, 84, 89, 91]
)

results = peft_trainer.predict(items_for_manual_review)

df = pd.DataFrame(
    {
        "sms": [item["text"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df